In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

#import numpy as np # linear algebra
#import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from google.cloud import bigquery
import pandas as pd

In [3]:
client = bigquery.Client()

Using Kaggle's public dataset BigQuery integration.


In [4]:
query1 = """
SELECT 
  SUM(value/POWER(10,18)) AS sum_tx_ether,
  AVG(gas_price*(receipt_gas_used/POWER(10,18))) AS avg_tx_gas_cost,
  DATE(timestamp) AS tx_date
FROM
  `bigquery-public-data.crypto_ethereum.transactions` AS transactions,
  `bigquery-public-data.crypto_ethereum.blocks` AS blocks
WHERE TRUE
  AND transactions.block_number = blocks.number
  AND receipt_status = 1
  AND value > 0
GROUP BY tx_date
HAVING tx_date >= '2018-01-01' AND tx_date <= '2018-12-31'
ORDER BY tx_date
"""
query = """
SELECT 
  from_address,
  to_address,
  value
FROM
  `bigquery-public-data.crypto_ethereum.transactions`
WHERE value > 0
"""

In [5]:
def estimate_gigabytes_scanned(query, client):
    # see https://cloud.google.com/bigquery/docs/reference/rest/v2/jobs#configuration.dryRun
    my_job_config = bigquery.job.QueryJobConfig()
    my_job_config.dry_run = True
    my_job = client.query(query, job_config=my_job_config)
    BYTES_PER_GB = 2**30
    return my_job.total_bytes_processed / BYTES_PER_GB

In [6]:
#print('Estimated size: {:.2f} GB'.format(estimate_gigabytes_scanned(query, client)))

In [7]:
query_job = client.query(query)

In [8]:
START_INDEX = 0
MAX_RESULTS = 1000000
iterator = query_job.result(timeout=300, start_index=START_INDEX, max_results=MAX_RESULTS)
TOTAL_ROWS = iterator.total_rows
print('Iterator: {} rows'.format(TOTAL_ROWS))
#rows = list(iterator)
#print('Rows: done')

Iterator: 631391869 rows


In [9]:
from collections import defaultdict
from decimal import Decimal
import pickle
balances = defaultdict(Decimal)

In [10]:
i = 0
for row in iterator:
    value = row.get('value')
    balances[row.get('from_address')] -= value
    balances[row.get('to_address')] += value
    #i += 1
    #if not (i % 1000000):
    #    print('{} out of {}'.format(i, TOTAL_ROWS))

In [11]:
#from sys import getsizeof
#print(getsizeof(balances))

In [12]:
#balances
with open('data.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(balances, f, pickle.HIGHEST_PROTOCOL)

In [13]:
#df = iterator.to_dataframe(progress_bar_type='tqdm_notebook')

In [14]:
#import requests
#from bs4 import BeautifulSoup
#import csv

#URL = "https://etherscan.io/accounts"
#resp = requests.get(URL)
#sess = requests.Session()
#soup = BeautifulSoup(sess.get(URL).text, 'html.parser')